# Installation

To install these scripts I suggest doing the following:

1. Create folder in home directory called pybin. Open terminal then type 
    
    ```cd ```
    
    ```mkdir ~/pybin```

2. Clone this github repo into pybin. 
    
    ```cd pybin``` 
    
    ```git clone https://github.com/jsolvason/js```
    
3. Go back to your home directory 

    ```cd ~```

4. Find your bash profile (The profile will be called ```.bash_profile```,```.bashrc```, or ```.zshrc```).

    ```ls -a ~``` 

5. Open your bash profile with the terminal word processor (this assumes its named ```.zshrc```) 

    ```nano ~./zshrc```

6. Add the following line to your bash profile 

    ```export PYTHONPATH=~/pybin/js:~/pybin/:$PYTHONPATH```


7. Test this works by creating a new terminal window and typing the follwoing. If this does not return an error message, then it works!

    ```python```

    ```import js``` 
    
8. You will also need to install the python library ```Biopython``` to your conda environment to use some of my scripts. Activate the conda environment you will use ```js``` in and install the following conda package https://anaconda.org/anaconda/biopython


# Download reference files

Reference files are too large to load to github. They can be found on our labs google drive here.

https://drive.google.com/drive/u/1/folders/1EP85teWmxO69mGgdENufN6YEux7i3X33

# Load all packages

In [1]:
import js
import jsAff as jsa
import jsDna as jsd
import jsGenome as jsg
import jsMpra as jsm
import jsSeq as jss

# js

In [2]:
import js
help(js)

Help on module js:

NAME
    js

FUNCTIONS
    dprint(d, n=0)
        Print a dictionary
    
    flatten_list(l)
        Returns [1,2,3,4] for inputted [[1,2],[3,4]]
    
    get_basename(fn)
        Get basename of file. e.g. /path/to/file.suff1.txt => file
    
    get_todays_datestring(format='%Y%m%d')
        Get todays date as string formatted YYYYMMDD
    
    md5_comparison(md52fn1, title1, md52fn2, title2)
        Compare md5s of files at two locations to ensure successful datatransfer
    
    md5_comparison_from_filenames(fn1, parsetype1, title1, fn2, parsetype2, title2)
        Wrapper function to simply plug in two md5.txt files and compare md5 sums
    
    million(number)
        Get number in terms of million
    
    mkdir_if_dir_not_exists(out_dir)
        Make a directory only if that directory doesnt exist
    
    percent(number, rounding_digit=1)
        Get percent of fraction
    
    read_tsv(fn, pc, header, breakBool=False, sep='\t', pc_list=False)
        Rea

## Read tsv

In [3]:
# Define file name
fn='./test-data/test.tsv'

# Usually you will just use the parameters 'fn','pc' (printcols), 'header'

# Usually you want to start by viewing what column is at what pythonic index in the row
for row in js.read_tsv(fn, pc=True, header=True):
    pass

0 name
1 sequence
2 value



In [4]:
# Now that we know what row is at what index, we can load each row
for row in js.read_tsv(fn, pc=False, header=True):
    name,seq,val=row
    print(name,seq,val)

a ATG 1.0
b AAA 3.1


## Read bed


In [5]:
# Define file name
fn='./test-data/test.bed'

# This program assumes the bed file has no headers. Bed file headers start with a '#', so if you really want 
# to read in a bed file with headers you can just skip all rows with a '#' at the begining of each column.

for row in js.read_tsv(fn, pc=True, header=False):
    pass

0 chr1
1 500000
2 501000



In [6]:
# Now you can do things like iterate over all kmers for each entry in the bed file.
#   This is how we search for 8bp ETS binding sites.

# Load genome
chr2seq=jsg.pklLoadGenome('/Users/joe/code/ref/genomes/human/hg38/hg38.fa.chr2seq.pydict.pickle')

# Load Affinity Data
seq2aff=jsa.loadAff(ref='/Users/joe/code/ref/binding_affinity/ets/parsed_Ets1_8mers.txt')

# Read bed
for row in js.read_tsv(fn, pc=False, header=False):
    chrom,start,end=row
    start,end=int(start),int(end)
    
    seq=chr2seq[chrom][start:end]
    for kmer in jsd.get_kmers(seq,8):
        
        # Find one example of an ets binding site and leave
        if kmer[2:6] in ['GGAA','GGAT','TTCC','ATCC']:
            affinity = round(seq2aff[kmer],2)
            print(kmer,affinity)
            break
    break

GTATCCTC 0.16


# jsAffinity

This module allows you to load affinity reference files in the form of a dictionary with ```key=dna_sequence``` and ```value=affinity``` where max affinity is 1.0

In [7]:
import jsAff as jsa
help(jsa)

Help on module jsAff:

NAME
    jsAff

FUNCTIONS
    loadAff(ref)
        Load an arbitrary affinity dataset. First column should be 8mer DNA 
        sequence and second column should be the normalized affinity (between 0-1).

DATA
    __warningregistry__ = {'version': 99}
    etsCores = {'ATCC', 'GGAA', 'GGAT', 'TTCC'}
    gataCores = {'GATA', 'TATC'}

FILE
    /Users/joe/Farley Lab Dropbox/Joe/bin/js/jsAff.py




## Downloading affinity files

Download here: https://drive.google.com/drive/u/1/folders/1DhCFV5_naMZRFjfwdsYMnGatXaTX824y

## Loading dictionary

In [8]:
# Load Ets1 affinity data
seq2aff=jsa.loadAff(ref='/Users/joe/code/ref/binding_affinity/ets/parsed_Ets1_8mers.txt')

In [9]:
# Note that all DNA 8mers measured, even thouse which are not real binding sites
js.dprint(seq2aff,0)

AAAAAAAA 0.14700431859740803


In [10]:
# Max and min affinities
round(min(seq2aff.values()),3),max(seq2aff.values())

(0.071, 1.0)

In [11]:
# Max sequence. 
seq2aff['CCGGAAGT']

1.0

In [12]:
# Note that you can search for fwd or rev and get same answer.
seq2aff[jsd.revcomp('CCGGAAGT')]

1.0

# jsDna

This module allows you to do various operations on DNA sequence

In [13]:
import jsDna as jsd
help(jsd)

Help on module jsDna:

NAME
    jsDna

FUNCTIONS
    GenerateAllPossibleSequences(template)
        Takes a string with letters A,T,C,G,N and returns all possible DNA strings converting N=>A,T,G,C
    
    GenerateRandomDNA(length)
        Takes length as input and returns a random DNA string of that length.
    
    GenerateSingleRandomSequence(template)
        Takes a string with letters A,T,C,G,IUPAC and returns a DNA string with rnadom AGTC nucleotides at position(s) with N.
    
    IupacToAllPossibleSequences(dna)
        Takes DNA with IUPAC letters and returns all possible DNA strings with only A/G/T/C.
    
    IupacToRegexPattern(dna)
        Takes DNA with IUPAC letters and returns a regex object that can search DNA with only A/T/G/C for the corresponding IUPAC DNA string.
    
    count_with_revcomp(pattern, seq)
    
    count_with_revcomp_re(re_pattern, length, seqFwd)
        A method to search dna sequence using re expression allowing for overlaps.
    
    gc_content(

## IUPAC Definitions

In [14]:
jsd.Iupac2AllNt

{'A': ['A'],
 'C': ['C'],
 'G': ['G'],
 'T': ['T'],
 'R': ['A', 'G'],
 'Y': ['C', 'T'],
 'S': ['G', 'C'],
 'W': ['A', 'T'],
 'K': ['G', 'T'],
 'M': ['A', 'C'],
 'B': ['C', 'G', 'T'],
 'D': ['A', 'G', 'T'],
 'H': ['A', 'C', 'T'],
 'V': ['A', 'C', 'G'],
 'N': ['A', 'C', 'G', 'T']}

## Reverse Complement

In [15]:
dna='ATGC'
jsd.revcomp(dna)    

'GCAT'

## Count a DNA pattern & its reverse complement

In [16]:
pattern='GGAA'
seq='AAAGGAATTTGGAACCCTTCCTT'
jsd.count_with_revcomp(pattern,seq)

3

## Iterating over kmers

In [17]:
string='AATTGGCC'
k=3
for kmer in jsd.get_kmers(string, k): print(kmer)

AAT
ATT
TTG
TGG
GGC
GCC


## Generating random DNA

In [18]:
length=5
jsd.GenerateRandomDNA(length)    

'GGCTC'

In [19]:
template='AANAA'
jsd.GenerateSingleRandomSequence(template)    

'AAGAA'

In [20]:
template='AANAA'
jsd.GenerateAllPossibleSequences(template)

{'AAAAA', 'AACAA', 'AAGAA', 'AATAA'}

In [21]:
dna='AYN'
jsd.IupacToAllPossibleSequences(dna)    

['ACA', 'ACC', 'ACG', 'ACT', 'ATA', 'ATC', 'ATG', 'ATT']

In [22]:
dna='AYN'
pattern=jsd.IupacToRegexPattern(dna)    
pattern,jsd.revcomp_regex(pattern)

('A(C|T)(A|C|G|T)', '(A|C|G|T)(A|G)T')

In [23]:
pattern='GGAW'
pattern=jsd.IupacToRegexPattern(pattern)  
seq='AAAGGAATTTGGAACCCTTCCTT'
jsd.count_with_revcomp_re(pattern,4,seq)

3

## Hamming Distance

In [24]:
str1='AATTGGCC'
str2='TTTTGGCC'
jsd.hamming(str1, str2)    

2

## GC Content

In [25]:
seq='ATGGCCAT'
jsd.gc_content(seq)    

0.5

# jsGenome

This module is used to load a genome as a dictionary with ```key=chromosome_name``` and ```value=chromosome_sequence```.

In [26]:
import jsGenome as jsg
help(jsg)

Help on module jsGenome:

NAME
    jsGenome

FUNCTIONS
    choose_random_chrom_pos(genomeLen, Chrom2NumStartEnd)
    
    faLoadGenome(file_genome)
        Loads arbitrary genome
    
    generate_Chrom2NumStartEnd(chr2seq)
        Generates obj for choose_random_chrom_pos()
    
    getLiftoverObject(startGenome, endGenome)
        A method to create a liftover object for jsg.liftover()
    
    liftover_interval_batch(in_bed, loObject, out_bed)
        A method to liftover intervals (chrom,start,end) from an entire bed file
    
    liftover_pos(liftOverObject, c, s)
        A method to liftover a single chrom/pos pair
    
    liftover_start_end(liftOverObject, c, s, e)
        A method to liftover a chrom/start/end.
    
    parse_cse(cse)
        A method to convert genome broser coords (ie chr1:1,000,000-1,200,000) to python variables.
    
    pklLoadGenome(pickled_file_genome)
        Loads arbitrary genome

FILE
    /Users/joe/Farley Lab Dropbox/Joe/bin/js/jsGenome.py




## Change coordinates from browser format to python variables

In [27]:
# def parse_cse(cse):
#     '''A method to convert genome broser coords (ie chr1:1,000,000-1,200,000) to python variables.'''
#     c,se=cse.split(':')
#     s,e=[int(i.replace(',','')) for i in se.split('-')]
#     return c,s,e
    
jsg.parse_cse('chr8:11,617,341-11,618,340')

('chr8', 11617341, 11618340)

## Download genomes to your computer

Found here https://drive.google.com/drive/u/1/folders/1zRZUe7bcVZqB1qsYbjWSAHIAlGw-UyAb

## Load genome

In [28]:
# Fastest method is to load binary python object saved
chr2seq=jsg.pklLoadGenome('/Users/joe/code/ref/genomes/human/hg38/hg38.fa.chr2seq.pydict.pickle')

In [30]:
# You can also load fasta file as genome
# chr2seq=jsg.faLoadGenome('/Users/joe/code/ref/genomes/human/hg38/hg38.fa')

In [31]:
# Inspect keys of dictionary
list(chr2seq.keys())[:5]

['chr1', 'chr10', 'chr11', 'chr11_KI270721v1_random', 'chr12']

In [32]:
# print random location in genome
chr2seq['chr1'][500000:500100]

'AGGTATCCTCTCATCTCAGCTTCCCTAGTAGTTGGAACTCTAGGTGCACAACACCACACCAGTTATTATTATTATTTTTTAATTTTTTATAGAGACAGGT'

## Get random genome coordinate

In [33]:
import jsGenome as jsg
genomeLen,Chrom2NumStartEnd=jsg.generate_Chrom2NumStartEnd(chr2seq)
jsg.choose_random_chrom_pos(genomeLen,Chrom2NumStartEnd)

('chr3', 130118091)

## Liftover coordinates from old genome to new genome

In [34]:
# Create liftover mapping
lo=jsg.getLiftoverObject('hg19','hg38')

In [35]:
# Liftover sungle position
jsg.liftover_pos(lo,'chr1',1000001)

('chr1', 1064621)

In [36]:
# Liftover start/end of interval
jsg.liftover_start_end(lo,'chr1',1000001,1000011)

('chr1', 1064621, 1064631)

In [37]:
# Liftover a whole file of start/ends (ie bed file)
# jsg.liftover_interval_batch(in_bed,loObject,out_bed)